In [15]:
import numpy as np
import gzip

In [8]:
def convolution(image,filt,bias,s=1):
    
    (n_f,n_c_f,f,_) = filt.shape
    n_c,in_dim,_ = image.shape
    
    out_dim = int((in_dim - f)/s) + 1
    
    assert n_c==n_c_f
    
    out = np.zeros((n_f,out_dim,out_dim))
    
    for curr_f in range(n_f):
        curr_y = out_y = 0
        
        while curr_y + f <= in_dim:
            curr_x = out_x = 0
            
            while curr_x + f<=in_dim:
                out[curr_f,out_y,out_x] = np.sum(filt[curr_f] * image[:,curr_y:curr_y + f,curr_x:curr_x + f]) + bias[curr_f]
                curr_x +=s
                out_x += 1
            
            curr_y +=s
            out_y +=1
    
    return out

In [9]:
def max_pooling(image,f=2,s=2):
    
    n_c,h_prev,w_prev = image.shape
    
    h = int((h_prev-f)/s) + 1
    w = int((w_prev - f)/s) + 1
    
    downsampled = np.zeros((n_c,h,w))
    
    for i in range(n_c):
        curr_y = out_y = 0
        
        while curr_y + g <=h_prev:
            curr_x = out_x = 0
            
            while curr_x + f <=w_prev:
                
                downsampled[i,out_y,out_x] = np.max(image[i,curr_y:curr_y+f,curr_x:curr_x + f])
                curr_x +=s
                out_x +=1
                
            curr_y +=s
            out_y +=1
            
    
    return downsampled

In [10]:
def softmax(raw_pred):
    out = np.exp(raw_pred)
    return out/np.sum(out)

In [11]:
def lossFunction(prob,label):
    return -np.sum(label*np.log(prob))

In [16]:
def extract_data(filename,num_images,IMAGE_WIDTH):
    print('Extracting',filename)
    
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(IMAGE_WIDTH*IMAGE_WIDTH*num_images)
        data = np.frombuffer(buf,dtype=np.uint8).astype(np.float32)
        data = data.reshape(num_images,IMAGE_WIDTH*IMAGE_WIDTH)
        
        return data

In [17]:
def extract_label(filename,num_images):
    print('Extracting',filename)
    
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1*num_images)
        labels = np.frombuffer(buf,dtype=np.uint8).astype(np.int64)
        
    return labels

In [21]:
def filterInitializer(size,scale = 1.0):
    
    stddev = scale/np.sqrt(np.prod(size))
    return np.random.normal(scale=stddev,size=size)

In [22]:
def weightInitializer(size):
    return np.random.standard_normal(size=size)*0.01